In [1]:
import re

def addEscapeSequence(math_exp):
    chars = ['.', '^', '$', '*', '+', '?','(', ')', '\\', '[', '{', '|', '-', ']']
    new_exp = ''
    for ch in math_exp:
        if ch in chars:
            new_exp += '\\' + ch
        else:
            new_exp += ch
    return new_exp
        
        
print(addEscapeSequence('\\frac{Q}{D}'))

\\frac\{Q}\{D}


In [2]:
# def replaceNumbers(math_exp):
#     new_exp = ''
#     for idx in range(len(math_exp)):
#         if (math_exp[idx].isdigit() == False):
#             new_exp += math_exp[idx]
#         elif(math_exp[idx].isdigit() == True):
#             if((idx == len(math_exp)-1) or (math_exp[idx+1].isdigit() == False)):
#                 new_exp += '(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))'
#             else:
#                 continue
#     return new_exp
    
# print(replaceNumbers(addEscapeSequence('a^{2}+b^{2}')))            

In [3]:
def replaceNumbers(math_exp):
    new_exp = ''
    regex = re.compile(r'(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))')
    new_exp = re.sub(regex, '(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))', math_exp)
    return new_exp

print(replaceNumbers(addEscapeSequence('a^{2}+b^{2}')))

a\^\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}\+b\^\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}


In [4]:
def replaceVariables(math_exp):
    new_exp = ''
    regex = re.compile(r'[a-zA-Z]+')
    new_exp = re.sub(regex, '[a-zA-Z]+', math_exp)
    return new_exp

print(replaceVariables(replaceNumbers(addEscapeSequence('a^{2}+b^{2}'))))

[a-zA-Z]+\^\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}\+[a-zA-Z]+\^\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}


In [5]:
def replaceNonTags(math_exp):
    return replaceVariables(replaceNumbers(addEscapeSequence(math_exp)))

In [6]:
def processTagsAndVariables(math_exp):
    tag_set = set(['frac', 'tan', 'log_2', 'tanh'])
    new_exp = ''
    breaks = [0]
    
    for idx in range(len(math_exp)):
        if(math_exp[idx] == '\\'):
            breaks.append(idx)
            
    breaks.append(len(math_exp))
    
    for idx in range(len(breaks)-1):
        start = breaks[idx]
        end = breaks[idx+1]
        sub_exp = math_exp[start:end]
        start = 0
        new_sub_exp = ''
        if(sub_exp == '\\'):
            new_exp += sub_exp
            continue
        elif(len(sub_exp) and sub_exp[0] == '\\'):
            new_sub_exp += '\\'
            start = 1
        found = False
        for pos in reversed(range(start, len(sub_exp))):
            if(sub_exp[start:pos+1] in tag_set):
                found = True
                new_sub_exp += sub_exp[:pos+1]+replaceNonTags(sub_exp[pos+1:])
                break
        if(not found):
            new_sub_exp = replaceVariables(sub_exp)
        new_exp += new_sub_exp
    
    return new_exp
            
                
             

math_exp = 'whole\\frac{\\tanh{45}}{\\log_2{denominator}}'
print(processTagsAndVariables(math_exp))

[a-zA-Z]+\\frac\{\\tanh\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}}\{\\log_2\{[a-zA-Z]+}}


In [7]:
#abstraction

In [8]:
def create_all_regexes(input_regex, regexes):
    stack = []
    for i in range(len(input_regex)):
        if (input_regex[i:i+2] == '\\{'):
            stack.append(i+1)
        elif (input_regex[i] == '}'):
            new_regex = input_regex[0:stack[-1]+1]+'.+?'+input_regex[i:]
            if new_regex in regexes:
                return
            regexes.add(new_regex)
            create_all_regexes(new_regex, regexes)
            del stack[-1]


In [10]:
og = processTagsAndVariables(math_exp)
all_regex = set()
all_regex.add(og)
create_all_regexes(og, all_regex)
for regex in all_regex:
    print(regex)

[a-zA-Z]+\\frac\{\\tanh\{.+?}}\{.+?}
[a-zA-Z]+\\frac\{.+?}\{\\log_2\{.+?}}
[a-zA-Z]+\\frac\{.+?}\{\\log_2\{[a-zA-Z]+}}
[a-zA-Z]+\\frac\{\\tanh\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}}\{\\log_2\{.+?}}
[a-zA-Z]+\\frac\{.+?}\{.+?}
[a-zA-Z]+\\frac\{\\tanh\{.+?}}\{\\log_2\{.+?}}
[a-zA-Z]+\\frac\{\\tanh\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}}\{.+?}
[a-zA-Z]+\\frac\{\\tanh\{.+?}}\{\\log_2\{[a-zA-Z]+}}
[a-zA-Z]+\\frac\{\\tanh\{(((\+|-)?([0-9]+)(\.[0-9]+)?)|((\+|-)?\.?[0-9]+))}}\{\\log_2\{[a-zA-Z]+}}
